In [18]:
import torch
import transformers
from openai import OpenAI
from app_config import get_config
import json
import logging
from datetime import datetime, timezone

In [19]:
with open("models/data/eg051_01_01_de_large-v3_de_unprocessed.json", "r", encoding="utf-8") as f:
    # Load the whisper results from the JSON file
    whisper_results = json.load(f)
segments = whisper_results["segments"]

In [20]:
def json_dictionary_parser(llm_output):
    """Parse a JSON string and return a dictionary."""
    try:
        parsed_llm_output = json.loads(llm_output)
    except json.JSONDecodeError as e:
        logging.error(f"Error parsing the JSON output: {e}")
        parsed_llm_output = {}
    
    if not isinstance(parsed_llm_output, dict):
        logging.warning("LLM output was not a dictionary. Returning empty dictionary.")
        parsed_llm_output = {}

    return parsed_llm_output

"""Summarize the given text using KISSKI API."""

api_key = "08f9793910421b5a38ac2f563d41028b"
base_url = "https://chat-ai.academiccloud.de/v1"
model = "qwen2.5-vl-72b-instruct" # Choose any available model
# model = "llama-3.3-70b-instruct" # Choose any available model

    # Start OpenAI client
client = OpenAI(
api_key = api_key,
base_url = base_url
)
prompt_system = (
    "Du bist Forscher und Digital Humanist, der folgende harte biografische Fakten aus einem Oral History-Interview in einer JSON-Datei speichert:\n"
    "1) Wie lautet der Vorname der interviewten Person?\n"
    "2) Wie lautet der Nachname der interviewten Person?\n"
    "3) Falls die Person einen vom Nachnamen abweichenden Geburtsnamen hat: Wie lautet der Geburtsname der interviewten Person?\n"
    "4) Hat de Person weitere Nachnamen?\n"
    "5) Hat die Person weitere Vornamen?\n"
    "6) Wie lautet das Geburtsdatum der interviewten Person?\n"
    "7) Wie lässt sich die Person mit einem Satz beschreiben?\n"
    "8) Welches Geschlecht hat die interviewte Person?\n"
    "9) Fasse die Biogafie des Interviewten als Fließtext zusammen in einem Absatz zusammen.\n"
    "Die Informationen sollen in einem JSON-Format ausgegeben werden, das folgende Struktur hat und keine zusätzlichen Kommentare, Erklärungen oder Formatierungszeichen wie Backticks ('''/```) enthält:\n"
    "{\n"
    "  'Vorname': 'Vorname',\n"
    "  'Nachname': 'Nachname',\n"
    "  'Geburtsname': 'Geburtsname',\n"
    "  'Weitere Namen': ['weiterer Nachname1', 'weiterer Nachname2'],\n"
    "  'Weitere Vornamen': ['weiterer Vorname1', 'weiterer Vorname2'],\n"
    "  'Geburtsdatum': 'Geburtsdatum',\n"
    "  'Personenbeschreibung': 'Beschreibung der Person',\n" 
    "  'Geschlecht': 'Geschlecht'\n"
    "  'Biografie': 'Fließtext über die Biografie der Person'\n"
    "Noch einmal: Gib ausschließlich gültiges JSON zurück. Keine zusätzlichen Kommentare, Erklärungen oder Formatierungszeichen wie Backticks ('''/```).\n"
)

prompt_content = ""
for segment in segments:
    prompt_content += f"{segment['text']}\n"
    
# Get response
chat_completion = client.chat.completions.create(
messages=[{"role":"system","content":prompt_system},{"role":"user","content": prompt_content}],
model= model,
)
response = chat_completion.choices[0].message.content
print(response)

# Parse response as JSON
parsed_response = json_dictionary_parser(response)

trials = 0
while not parsed_response and trials < 3:
    trials += 1
    logging.warning("LLM response was not a valid JSON. Retrying with correction prompt.")
    #Correct the output via LLM to ensure it is a dictionary
    prompt_correction = (
        "Die Antwort des LLM war kein gültiges JSON-Format. Bitte überprüfe deine Antwort und gib sie im korrekten JSON-Format zurück.\n"
        "Hier ist die Antwort, die korrigiert werden muss:\n"
        f"{response}\n"
        "Gib ausschließlich gültiges JSON zurück. Keine zusätzlichen Kommentare, Erklärungen oder Formatierungszeichen wie Backticks ('''/```).\n"
    )
    chat_completion_correction = client.chat.completions.create(
        messages=[{"role":"system","content":prompt_system},{"role":"user","content": prompt_correction}],
        model= model,
    )
    corrected_response = chat_completion_correction.choices[0].message.content

    parsed_response = json_dictionary_parser(corrected_response)

print(parsed_response)

{
  "Vorname": "Nikolaus",
  "Nachname": "Apostolopoulos",
  "Geburtsname": "",
  "Weitere Namen": [],
  "Weitere Vornamen": ["Alexander"],
  "Geburtsdatum": "1948-09-25",
  "Personenbeschreibung": "Griechischer Hochschulprofessor und Expert für digitale Systeme an der Freien Universität Berlin, der sich mit der Implementation technischer Lösungen im akademischen Bereich und in der Lehre beschäftigt.",
  "Geschlecht": "Mann",
  "Biografie": "Nikolaus Apostolopoulos wurde am 25. September 1948 in Athen geboren und kam 1967 als 18-Jähriger nach Berlin, um an der Freien Universität Berlin Wirtschaftswissenschaften zu studieren. Seine akademische Karriere an der Freien Universität umfasst die Entwicklung von digitalen Lernumgebungen und der Einbezug von Informationstechnologie in den akademischen Unterricht, insbesondere durch die Gründung des CEDIS (Center für Digitale Systeme) im Jahr 2003. Apostolopoulos erwarb einen akademischen Titel als Honorarprofessor und hat viele Jahre lang über 

# Export as JSON

In [21]:
now = datetime.now(tz=timezone.utc)
date_suffix = now.strftime(".%Y-%m-%dT_%H%M%SZ")
result_json = f"models/data/bio_data_eg051_{model}_{date_suffix}.json"
with open(result_json, "w", encoding="utf-8") as f:
    json.dump(parsed_response, f, indent=4, ensure_ascii=False)

# Export as XML